In [1]:
import matplotlib.pyplot as plt
import numpy as np
import astropy.constants as c
import astropy.units as u

## Apparent Magnitudes

In [2]:
magnitudes = {}

In [3]:
#TOI 237

magnitudes['TOI 237'] = {}

magnitudes['TOI 237']['d_pc'] = 38.11
magnitudes['TOI 237']['Tmag'] = 13.4102
magnitudes['TOI 237']['B'] = 17.718
magnitudes['TOI 237']['V'] = 16.37
magnitudes['TOI 237']['Ic'] = 13.4102
magnitudes['TOI 237']['J'] = 11.74
magnitudes['TOI 237']['H'] = 11.19
magnitudes['TOI 237']['K'] = 10.896
magnitudes['TOI 237']['Gaia_bp'] = 16.4447
magnitudes['TOI 237']['Gaia_rp'] = 13.5016
magnitudes['TOI 237']['GaiaG'] = 14.754

#TOI 122

magnitudes['TOI 122'] = {}

magnitudes['TOI 122']['d_pc'] = 62.12
magnitudes['TOI 122']['Tmag'] = 13.084
magnitudes['TOI 122']['B'] = 17.061
magnitudes['TOI 122']['V'] = 15.526
magnitudes['TOI 122']['Ic'] = 13.084
magnitudes['TOI 122']['J'] = 11.531
magnitudes['TOI 122']['H'] = 11.02
magnitudes['TOI 122']['K'] = 10.771
magnitudes['TOI 122']['Gaia_bp'] = 15.7971
magnitudes['TOI 122']['Gaia_rp'] = 13.1523
magnitudes['TOI 122']['GaiaG'] = 14.3357

#TOI 203

magnitudes['TOI 203'] = {}

magnitudes['TOI 203']['d_pc'] = 24.807
magnitudes['TOI 203']['Tmag'] = 12.2423
magnitudes['TOI 203']['B'] = 12.2423
magnitudes['TOI 203']['V'] = 15.017
magnitudes['TOI 203']['Ic'] = 12.2423
magnitudes['TOI 203']['J'] = 10.56
magnitudes['TOI 203']['H'] = 10.053
magnitudes['TOI 203']['K'] = 9.722
magnitudes['TOI 203']['Gaia_bp'] = 15.2793
magnitudes['TOI 203']['Gaia_rp'] = 12.3345
magnitudes['TOI 203']['GaiaG'] = 13.588

# TOIs 2269.01-0.3

magnitudes['TOI 2269'] = {}

magnitudes['TOI 2269']['d_pc'] = 82.1896
magnitudes['TOI 2269']['Tmag'] = 11.9521
magnitudes['TOI 2269']['B'] = 15.038
magnitudes['TOI 2269']['V'] = 13.476
magnitudes['TOI 2269']['Ic'] = 11.9521
magnitudes['TOI 2269']['J'] = 10.765
magnitudes['TOI 2269']['H'] = 10.107
magnitudes['TOI 2269']['K'] = 9.928
magnitudes['TOI 2269']['Gaia_bp'] = 13.8533
magnitudes['TOI 2269']['Gaia_rp'] = 11.9622
magnitudes['TOI 2269']['GaiaG'] = 12.9097

## Mann+ 2015 Table 2 Coefficients

For temperature calculations

In [4]:
BP_RP = np.array([3.245,-2.4309,1.043,-0.2127,0.01649,0,0])
V_J = np.array([2.840,-1.3453,0.3906,-0.0546,0.002913,0,0])
V_Ic = np.array([2.455,-1.5701,0.6891,-0.1500,0.01254,0,0])
r_z = np.array([1.547,-0.7053,0.3656,-0.1008,0.01046,0,0])
r_J = np.array([2.445,-1.2578,0.4340,-0.0720,0.004502,0,0])

BP_RP_JH = np.array([3.172,-2.475,1.082,-0.2231,0.01738,0.08776,0.04355])
V_J_JH = np.array([2.769,-1.421,0.4284,-0.06133,0.003310,0.1333,0.05416])
V_Ic_JH = np.array([1.568,-0.4381,0.07749,-0.005610,0.0,0.2441,-0.09257])
r_z_JH = np.array([1.384,-0.6132,0.3110,-0.08574,0.008895,0.1865,-0.02039])
r_J_JH = np.array([2.151,-1.092,0.3767,-0.06292,0.003950,0.1697,0.03106])

## Mass Equations

Mann+ 2019, table 7

In [5]:
def Mstar_Ks_Mann19(ID): # errors are 2.2%
    
    mK = magnitudes[ID]['K']
    dist_pc = magnitudes[ID]['d_pc']
    
    a = np.array([-0.647,-0.208,-8.43e-4,7.87e-3,1.42e-4,-2.13e-4])
    i = np.array([0,1,2,3,4,5])
    
    MK = mK - 5*np.log10(dist_pc/10)
    mass_integral = a*((MK-7.5)**i)
    Mass = np.sum(mass_integral)
    error = 0.022*(10**Mass)
    print(r'{:.4f}+/-{:.4f} Msun'.format(10**Mass,error))

## Radius Equations

Mann+ 2015, Eqn 4, table 1

In [6]:
def Rstar_Ks_Mann15(ID): # 2.89% error
    
    mK = magnitudes[ID]['K']
    dist_pc = magnitudes[ID]['d_pc']    
    
    a = 1.9515
    b = -0.352
    c = 0.0168
    MK = mK - 5*np.log10(dist_pc/10)
    radius = a + b*MK +c*MK**2.0
    
    error = 0.0289*radius
    print(r'{:.4f}+/-{:.4f} Rsun'.format(radius,error))

In [7]:
def Rstar_Teff_Mann15(Teff): # 13.4% error
        
    a = 10.544
    b = -33.7546
    c = 35.1909
    d = -11.5928
    
    X = T_eff/3500
    
    Radius = a + b*X + c*X**2 + d*X**3
    error = 0.14*Radius
    print(r'{:.4f}+/-{:.4f} Rsun'.format(Radius,error))

## Teff Equation

Mann+ 2015, Table 2

In [8]:
def Teff_Mann15(ID,coeffs=BP_RP_JH): # errors are ~80K or 100K

    BP = magnitudes[ID]['Gaia_bp']
    RP = magnitudes[ID]['Gaia_rp']
    V = magnitudes[ID]['V']
    J = magnitudes[ID]['J']
    H = magnitudes[ID]['H']
    Ic = magnitudes[ID]['Ic']
    
    if ((coeffs is BP_RP) or (coeffs is BP_RP_JH)):
        X = BP-RP
    if (coeffs is V_J) or (coeffs is V_J_JH):
        X = V-J
    if (coeffs is V_Ic) or (coeffs is V_Ic_JH) :
        X = V-Ic
        
    polynomial = np.array([1,X,X**2,X**3,X**4,(J-H),(J-H)**2])
    
    Teff = np.sum(coeffs*polynomial)*3500
    
    print('{:.0f}'.format(Teff))

In [17]:
Teff_Mann15('TOI 122',coeffs = BP_RP)
Teff_Mann15('TOI 122',coeffs = V_J)
Teff_Mann15('TOI 122',coeffs = V_Ic)

Teff_Mann15('TOI 122',coeffs = BP_RP_JH)
Teff_Mann15('TOI 122',coeffs = V_J_JH)
Teff_Mann15('TOI 122',coeffs = V_Ic_JH)

3442
3361
3471
3409
3305
3427


In [18]:
Temps = np.array([3442,3361,3471,3409,3305,3427])
errors = np.array([52,55,53,49,48,52])
errors = np.sqrt(errors*np.array([60,60,60,60,60,60]))

np.average(a = Temps, axis = None, weights = errors)

3403.020969779429

In [19]:
np.std(Temps)

54.95983381828345

In [20]:
Teff_Mann15('TOI 237',coeffs = BP_RP)
Teff_Mann15('TOI 237',coeffs = V_J)
Teff_Mann15('TOI 237',coeffs = V_Ic)

Teff_Mann15('TOI 237',coeffs = BP_RP_JH)
Teff_Mann15('TOI 237',coeffs = V_J_JH)
Teff_Mann15('TOI 237',coeffs = V_Ic_JH)

3289
3164
3212
3283
3139
3188


In [22]:
Temps = np.array([3289,3164,3212,3283,3139,3188])
errors = np.array([52,55,53,49,48,52])
errors = np.sqrt(errors*np.array([60,60,60,60,60,60]))

np.average(a = Temps, axis = None, weights = errors)
np.std(Temps)

56.53538714822779

In [10]:
Teff_Mann15('TOI 2269',coeffs = BP_RP)
Teff_Mann15('TOI 2269',coeffs = V_J)
Teff_Mann15('TOI 2269',coeffs = V_Ic)

Teff_Mann15('TOI 2269',coeffs = BP_RP_JH)
Teff_Mann15('TOI 2269',coeffs = V_J_JH)
Teff_Mann15('TOI 2269',coeffs = V_Ic_JH)

Rstar_Ks_Mann15('TOI 2269')
Mstar_Ks_Mann19('TOI 2269')

4026
3966
4198
4029
3966
4134
0.5485+/-0.0159 Rsun
0.5376+/-0.0118 Msun


## Luminosity 

Bolometric Corrections from Mann+ 2015, Table 3

In [41]:
def Lumin_BoloCorr_Mann15(ID): #Mann+ 2015
    
    mV = magnitudes[ID]['V']
    mJ = magnitudes[ID]['J']
    dist_pc = magnitudes[ID]['d_pc']
    
    #sigma = +/- 0.016, or 5% (inflated from 2-3%)
    
    a = 0.5817
    b = -0.4168
    c = -0.08165
    d = 4.084e-3
    
    X = mV-mJ
    Distance = dist_pc/10
    MV = mV - 5*np.log10(Distance)
    
    BCv = a + b*(X) + c*(X**2) + d*(X**3)
    mbol = mV + BCv
    Mbol = mbol - 5*np.log10(Distance)
    Msun = 4.8
    
    lum = 10**((Msun-Mbol)/2.5)
    
    error = 0.02*lum
    print(r'{:.5f}+/-{:.5f} Lsun'.format(lum,error))
    #return lum

In [44]:
def Lumin_Definition(R,Teff):
    
    L = R**2*(Teff/5770)**4
    return np.percentile(L, [16.,50.,84.])

In [45]:
Lumin_BoloCorr_Mann15('TOI 122')
Lumin_BoloCorr_Mann15('TOI 237')

0.01401+/-0.00028 Lsun
0.00409+/-0.00008 Lsun


In [48]:
Lumin_Definition(R=np.random.normal(0.211,0.006,2000),
                Teff = np.random.normal(3212,100,2000))

Lumin_Definition(R=np.random.normal(0.334,0.02,2000),
                Teff = np.random.normal(3403,100,2000))

array([0.01134056, 0.0134451 , 0.01577747])

## Density

In [3]:
def Density_Definition(M,R):
    
    Mass = M*c.M_sun * (1000*u.g/u.kg)
    volume = 4/3*np.pi*(R*c.R_sun)**3 * (100*u.cm/u.m)**3
    density = Mass/volume
    
    return density

## Plots

In [2]:
# plt.figure(figsize=(8,8))
# plt.xlim(0.1,0.5)
# plt.xlabel('Mass (Msun)')
# plt.ylim(0.1,0.5)
# plt.ylabel('Radius (Rsun)')

# plt.axvspan(0.17,0.188, label='Mann+ 2019 (MKs)',color='black',alpha=0.3)
# plt.axhspan((0.183),(0.203), label='Mann+ 2015 (Teff=3085K)',color='hotpink',alpha=0.3)
# plt.axhspan((0.2),(0.222), label='Mann+ 2015 (MKs)',color='b',alpha=0.3)
# plt.axhspan((0.24),(0.32), label='Magellan FIRE',color='orange', alpha=0.3)
# plt.axhspan((0.195),(0.219), label='Michele Silversteins Pipeline',color='green',alpha=0.3)

# plt.plot(Masses_from_Mann2019/c.M_sun,radii_from_density,color='k',
#          label=r'Radius based on $\rho_{*}$')

# plt.legend()
# plt.show()

In [3]:
# plt.figure(figsize=(8,8))
# plt.xlim(2800,3900)
# plt.xlabel('Teff (K)')
# plt.ylim(0.1,0.5)
# plt.ylabel('Radius (Rsun)')

# plt.axhspan((0.183),(0.203), label='Mann+ 2015 (Teff=3085K)',color='blue',alpha=0.3)
# plt.axhspan((0.2),(0.222), label='Mann+ 2015 (MKs)',color='red',alpha=0.3)

# plt.axhspan((0.24),(0.32), label='Magellan FIRE',color='orange', alpha=0.3)
# plt.axvspan((2998),(3172),color='orange', alpha=0.3)

# plt.axhspan((0.195),(0.219), label='Michele Silversteins Pipeline',color='green',alpha=0.3)
# plt.axvspan((3073),(3255),color='green', alpha=0.3)

# plt.errorbar()

# plt.axvspan((2900),(3100), label='SALT HRS',color='hotpink', alpha=0.3)
# plt.axvspan((2874),(3176), label='Mann+ 2015 (VJH)',color='k', alpha=0.3)

# plt.legend()
# plt.show()

In [34]:
# plt.figure(figsize=(8,8))
# plt.xlim(0.1,0.5)
# plt.xlabel('Mass (Msun)')
# plt.ylim(0.1,0.5)
# plt.ylabel('Radius (Rsun)')

# plt.axvspan(0.296,0.328, label='Mann+ 2019 (MKs)',color='black',alpha=0.3)
# plt.axhspan((0.483),(0.533), label='Mann+ 2015 (Teff=3732K)',color='hotpink',alpha=0.3)
# plt.axhspan(0.317,0.351, label='Mann+ 2015 (MKs)',color='b',alpha=0.3)
# plt.axhspan((0.41),(0.47), label='Magellan FIRE',color='orange', alpha=0.3)
# plt.axhspan((0.316),(0.354), label='Michele Silversteins Pipeline',color='green',alpha=0.3)

# plt.plot(Masses_from_Mann2019/c.M_sun,radii_from_density,color='k',
#          label=r'Radius based on $\rho_{*}$')

# plt.legend()
# plt.show()

In [35]:
# plt.figure(figsize=(8,8))
# plt.xlim(3000,4000)
# plt.xlabel('Teff (K)')
# plt.ylim(0.1,0.5)
# plt.ylabel('Radius (Rsun)')

# plt.axhspan((0.483),(0.533), label='Mann+ 2015 (Teff=3732K)',color='hotpink',alpha=0.3)
# plt.axhspan(0.317,0.351, label='Mann+ 2015 (MKs)',color='b',alpha=0.3)

# plt.axhspan((0.41),(0.47), label='Magellan FIRE',color='orange', alpha=0.3)
# plt.axvspan((3647),(3817), label='Magellan FIRE',color='orange')

# plt.axvspan((3600),(3800), label='SALT HRS',color='blue')

# plt.axvspan((3046),(3356), label='Mann+ 2015 (VJH)',color='purple')

# plt.axhspan((0.316),(0.354), label='Michele Silversteins Pipeline',color='green',alpha=0.3)
# plt.axvspan((3276),(3462),color='green')

# plt.legend()
# plt.show()